In [7]:
year1 =      "22"            # 만들 데이터의 전월 해 입력 ex) 20, 21, ...

mon1 =      "01"            # 만들 데이터의 전월 입력 ex) 03, 05, 08, 09, 10, 12 ,,,

year2 =     "22"           # 만들 데이터의 해 입력 ex) 20, 21, ...

mon2 =      "02"            # 만들 데이터의 현월 입력 ex) 03, 05, 08, 09, 10, 12 ,,,



In [8]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import openpyxl
import datetime
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta 
import datetime as dt 

os.chdir('C:/Users/tommy/Desktop/파이썬/가스 데이터/final_test/')
file_out2 = './result_data/20'+year2+'.'+mon2+'월_거래처용기수불_pre.xlsx'

# 전월재고만 뺴기

def last_m_stock(year,mon):
    df_r =pd.read_excel('./data/'+year+'.'+mon+'월 용기수불.xlsx', skiprows = 2, sheet_name='거래처별용기수불')
    df_r1 =pd.read_excel('./data/'+year+'.'+mon+'월 용기수불.xlsx', skiprows = 2, sheet_name='거래처별용기수불')
    try:
        df_r.drop(columns = {'Unnamed: 0'},inplace = True)
        df_r1.drop(columns = {'Unnamed: 0'},inplace = True)
    except:
        pass

    df_r1 = df_r1[list(df_r1.columns[list(map(lambda x : 'Unnamed' not in x, df_r1.columns))])]
    df_r1 = df_r1.rename(columns = {'                   품  명\n거 래 처':'Customer'})
    df_r1 = df_r1[(pd.notnull(df_r1['Customer']))]
    df_r1.columns = list(map(lambda x : x.upper(),df_r1.columns))
    df_r1.columns = list(map(lambda x : re.sub(" ","",x),df_r1.columns))

    #이름 정렬

    l1 = list(map(lambda x : x + "_" +'2_실병하차',df_r1.columns[2:]))
    l2 = list(map(lambda x : x + "_" +'3_공병상차',df_r1.columns[2:]))
    l3 = list(map(lambda x : x + "_" +'4_금월재고',df_r1.columns[2:]))

    df_r1.columns = list(map(lambda x : x.upper(),df_r1.columns))
    df_r1.columns = list(map(lambda x : re.sub(" ","",x),df_r1.columns))
    df_r1.columns = ['Customer'] + list(map(lambda x : x + "_" +'1_전월재고',df_r1.columns[1:]))


    df_r.iloc[0,0] = "거래처"
    df_r.columns = df_r.iloc[0,:]

    df_r2 = df_r[["거래처","금월재고"]].iloc[2:,:]
    df_r2.columns = df_r1.columns 

    # 2_실병하차 3_공병상차으로 setting
    for i in l3:
        df_r2[i]=0
    
    df_r2 = df_r2[pd.notnull(df_r2["Customer"])].reset_index(drop=True)
    df_r3 = df_r2[(list(df_r2.columns)[:2] + sorted(df_r2.columns[2:]))].fillna(0)  
    df_r3["number"] = year+mon 
    return(df_r3)

year = []
month = []
now = dt.datetime(int("20"+year1),int(mon1),1)
for i in range(1,9):
    delta = relativedelta(months=i) 
    diff = now - delta
    year.append(str(diff)[2:4])
    month.append(str(diff)[5:7])

a = last_m_stock(year = year1,mon=mon1)

for year,mon in zip(year,month):   
    b = last_m_stock(year,mon)
    a = pd.concat([a,b]).sort_values(by = "number",ascending = False).drop_duplicates("Customer").reset_index(drop=True)
a = a.fillna(0)

# 이름 달라진 기업 확인
def change_name(old,new):
    global a
    a["Customer"] = np.where(a["Customer"] == old,new,a["Customer"])
    
new_name = pd.read_excel("./data/change_name.xlsx")
for old,new in zip(list(new_name["old_name"]),list(new_name["new_name"])):
    change_name(old,new)
a = a.sort_values(by = "number",ascending = False).drop_duplicates("Customer").reset_index(drop=True)
a.drop(columns = ["number"]).to_excel(file_out2,index=None)  
